# Fine-tuning BERT for multi-label text classification

## 환경 설정

HuggingFace Transformers, Datasets 라이브러리를 설치합니다.

In [2]:
!pip install -q transformers datasets

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
import re
import os
import urllib.request
from tqdm import tqdm
from transformers import BertTokenizer, TFBertForSequenceClassification
from tensorflow.keras.utils import to_categorical

C:\Users\wjsrn\anaconda3\envs\py39t116\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## train test로 파일을 분리합니다.

In [2]:
import pandas as pd
df = pd.read_csv("./data/220712_sentiment_anlysis_result_data.csv", index_col=0)
df = df.sample(n=4800)
print(len(df))
df.head(3)

4800


,Date,Title,주가의 날짜,등락률(y),updown,Pos,Neg,Mid,감성지수
23046,2021-01-27,NNIP 유럽 주식 유망 신흥국도 5 15 수익률 가능,2021-01-28,-2.79,-1,1,0,0,1
353760,2022-04-18,이지스자산운용 현대건설 현대차와 MOU 도심 항공 모빌리티 구현 맞손,2022-04-19,1.50,1,1,0,0,1
396189,2022-06-22,마켓인 LG유플러스 공모채 수요예측 흥행 에 증액 발행,2022-06-23,-0.74,0,1,0,0,1


In [3]:
print('결측값 여부 :',df.isnull().values.any())

결측값 여부 : False


In [4]:
df['updown'].value_counts()

 0    1903
 1    1572
-1    1325
Name: updown, dtype: int64

In [5]:
train_df = df.sample(n=len(df)-800, random_state = 123, replace = False)
test_df = df.sample(n=800, random_state = 123, replace = False)

In [6]:
print(train_df.shape)
print(test_df.shape)

(4000, 9)
(800, 9)


In [7]:
train_df.to_csv("./data/train_3.csv",encoding='utf-8-sig',index = False)
test_df.to_csv("./data/test_3.csv",encoding='utf-8-sig',index = False)

## 미리 나눠놓은 train.csv, test.csv 파일을 불러옵니다.

In [8]:
### Q1. 데이터 파일을 지정하는 코드를 작성하시오
## 각각 train_data, test_data 의 변수에 지정하는 코드를 작성해 보시오

train_data = './data/train_3.csv'
test_data = './data/train_3.csv'

### datasets 를 이용해 train, test 파일을 로드합니다.

In [9]:
from datasets import load_dataset, ReadInstruction

dataset = load_dataset('csv', data_files={'train': train_data, 'test': test_data})

Using custom data configuration default-893da8bfc1199044


Extracting data files: 100%|████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 999.71it/s]


Dataset csv downloaded and prepared to C:\Users\wjsrn\.cache\huggingface\datasets\csv\default-893da8bfc1199044\0.0.0\51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58. Subsequent calls will reuse this data.


100%|███████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 181.78it/s]


### train과 test가 로드 된것을 확인합니다.
과제를 수행할 때에는 train / validataion / test 데이터로 분리해서 사용합니다.

In [10]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Date', 'Title', '주가의 날짜', '등락률(y)', 'updown', 'Pos', 'Neg', 'Mid', '감성지수'],
        num_rows: 4000
    })
    test: Dataset({
        features: ['Date', 'Title', '주가의 날짜', '등락률(y)', 'updown', 'Pos', 'Neg', 'Mid', '감성지수'],
        num_rows: 4000
    })
})

train 데이터의 첫 번째 데이터를 확인해봅니다.

In [11]:
example = dataset['train'][500]
example

{'Date': '2021-06-08',
 'Title': '금값 상승에도  사지 말고 팔아라  위험자산 권하는 까닭',
 '주가의 날짜': '2021-06-09',
 '등락률(y)': -1.46,
 'updown': -1,
 'Pos': 1,
 'Neg': 1,
 'Mid': 0,
 '감성지수': 0}

## Preprocess data

말씀드렸듯이 BERT는 텍스트가 아닌 `input_ids`를 입력으로 받습니다. 이를 위해 우선 훈련된 토크나이저를 활용해 텍스트를 토큰화합니다. 이번 예시에서는 `BertTokenizer`를 활용해 토큰화를 진행합니다.

In [14]:
from transformers import AutoTokenizer, BertTokenizer
import numpy as np

tokenizer = BertTokenizer.from_pretrained('klue/bert-base')

def convert_examples_to_features(examples, labels, max_seq_len, tokenizer):

    input_ids, attention_masks, token_type_ids, data_labels = [], [], [], []

    for example, label in tqdm(zip(examples, labels), total=len(examples)):
        # input_id는 워드 임베딩을 위한 문장의 정수 인코딩
        input_id = tokenizer.encode(example, max_length=max_seq_len, pad_to_max_length=True)

        # attention_mask는 실제 단어가 위치하면 1, 패딩의 위치에는 0인 시퀀스.
        padding_count = input_id.count(tokenizer.pad_token_id)
        attention_mask = [1] * (max_seq_len - padding_count) + [0] * padding_count

        # token_type_id은 세그먼트 인코딩
        token_type_id = [0] * max_seq_len

        assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
        assert len(attention_mask) == max_seq_len, "Error with attention mask length {} vs {}".format(len(attention_mask), max_seq_len)
        assert len(token_type_id) == max_seq_len, "Error with token type length {} vs {}".format(len(token_type_id), max_seq_len)

        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)
        data_labels.append(label)

    input_ids = np.array(input_ids, dtype=int)
    attention_masks = np.array(attention_masks, dtype=int)
    token_type_ids = np.array(token_type_ids, dtype=int)

    data_labels = np.asarray(data_labels, dtype=np.int32)

    return (input_ids, attention_masks, token_type_ids), data_labels

In [15]:
train_X, train_y = convert_examples_to_features(dataset['train']['Title'], dataset['train']['updown'], max_seq_len=128, tokenizer=tokenizer)

  0%|                                                                                         | 0/4000 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
C:\Users\wjsrn\anaconda3\envs\py39t116\lib\site-packages\transformers\tokenization_utils_base.py:2301: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (

In [16]:

test_X, test_y = convert_examples_to_features(dataset['test']['Title'],  dataset['test']['updown'], max_seq_len=128, tokenizer=tokenizer)

100%|████████████████████████████████████████████████████████████████████████████| 4000/4000 [00:00<00:00, 7186.85it/s]


In [17]:
input_id = train_X[0][0]
attention_mask = train_X[1][0]
token_type_id = train_X[2][0]
label = train_y[0]

print('단어에 대한 정수 인코딩 :',input_id)
print('어텐션 마스크 :',attention_mask)
print('세그먼트 인코딩 :',token_type_id)
print('각 인코딩의 길이 :', len(input_id))
print('정수 인코딩 복원 :',tokenizer.decode(input_id))
print('레이블 :',label)

단어에 대한 정수 인코딩 : [    2  1423  2483  2024  5126 25305  4376  8380  4115  4071  2116 22196
  2434  2024  2250  2122     3     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0]
어텐션 마스크 : [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [18]:
train_y = to_categorical(train_y, 3)
test_y = to_categorical(test_y, 3)

In [19]:
train_y

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.]], dtype=float32)

## 모델 정의

모델을 정의하기 위해 `transformers`에서 제공하는 `AutoModelForSequenceClassification` 모듈을 import합니다. pretrained된 모델의 weight를 로드하면 multi-label-classification을 위한 layer가 random한 weight로 생성됩니다.

In [38]:
model = TFBertForSequenceClassification.from_pretrained("klue/bert-base", num_labels=3, from_pt=True)
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.categorical_hinge
model.compile(optimizer=optimizer, loss=loss, metrics = ['accuracy'])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## 모델 훈련

We are going to train the model using HuggingFace's Trainer API. This requires us to define 2 things: 
허깅페이스에서 제공하는 Trainer API를 사용해 훈련을 시킵니다. 이를 위해 `TrainingArguments`와 `Trainer`를 정의해줘야 합니다.

* `TrainingArguments` : [문서](https://huggingface.co/transformers/main_classes/trainer.html#trainingarguments)
* `상세` : [문서](https://huggingface.co/docs/transformers/v4.20.0/en/main_classes/trainer#transformers.TrainingArguments)
* `Trainer` : [참조](https://huggingface.co/transformers/main_classes/trainer.html#id1).

In [39]:
model.fit(train_X, train_y, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
100/100 [==============================] - 55s 493ms/step - loss: 1.1093 - accuracy: 0.3684 - val_loss: 1.0717 - val_accuracy: 0.4038
Epoch 2/10
100/100 [==============================] - 48s 478ms/step - loss: 1.0879 - accuracy: 0.3622 - val_loss: 1.0342 - val_accuracy: 0.3575
Epoch 3/10
100/100 [==============================] - 48s 477ms/step - loss: 1.0609 - accuracy: 0.4141 - val_loss: 1.1078 - val_accuracy: 0.2925
Epoch 4/10
100/100 [==============================] - 48s 483ms/step - loss: 0.9830 - accuracy: 0.5041 - val_loss: 1.2225 - val_accuracy: 0.4075
Epoch 5/10
100/100 [==============================] - 48s 479ms/step - loss: 0.7467 - accuracy: 0.6522 - val_loss: 1.2507 - val_accuracy: 0.3775
Epoch 6/10
100/100 [==============================] - 48s 479ms/step - loss: 0.5087 - accuracy: 0.7728 - val_loss: 1.4398 - val_accuracy: 0.3875
Epoch 7/10
100/100 [==============================] - 48s 478ms/step - loss: 0.3000 - accuracy: 0.8753 - val_loss: 1.5944 - val_ac

In [40]:
model.evaluate(test_X, test_y, batch_size=1024)

4/4 [==============================] - 17s 4s/step - loss: 0.4229 - accuracy: 0.8572


[0.42290088534355164, 0.8572499752044678]

In [41]:
pred = model.predict(test_X)

125/125 [==============================] - 18s 138ms/step


In [42]:
logits = pred.logits
predictions = np.zeros(logits.shape)
predictions[np.where(logits >= 0.5)] = 1
correct_count = (predictions == test_y).sum().item()/ 3
acc = correct_count / len(test_y) * 100
print(acc)

89.23333333333335


In [43]:
for i in range(len(test_X[0])):
    print('정수 인코딩 복원 :',tokenizer.decode(test_X[0][i]))
    print('예측값: {} 결과값: {}'.format(predictions[i], test_y[i]))

정수 인코딩 복원 : [CLS] 엔씨소프트 리니지 장기 흥행 지속 목표가상승감소신영 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
예측값: [1. 0. 0.] 결과값: [1. 0. 0.]
정수 인코딩 복원 : [CLS] 에이디엠코리아 종속사 에이디엠바이오사이언스 주식 취득 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PA

예측값: [1. 0. 0.] 결과값: [1. 0. 0.]
정수 인코딩 복원 : [CLS] 공모주 흥행 랠리 핑거 공모가 1만6000원 수요예측 호조 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
예측값: [0. 1. 0.] 결과값: [0. 1. 0.]
정수 인코딩 복원 : [CLS] 인피니트헬스케어 지난해 영업익 74억 전년비 1 8 상승 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [

정수 인코딩 복원 : [CLS] PI첨단소재 최대주주 매각주간사 선정 지분 매각 검토중 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
예측값: [0. 1. 0.] 결과값: [0. 1. 0.]
정수 인코딩 복원 : [CLS] 코스닥 9 66p 0 94 내린 1 022 22 장종료 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] 

예측값: [0. 1. 0.] 결과값: [0. 1. 0.]
정수 인코딩 복원 : [CLS] 엔저 가속 6년 만에 달러당 123엔대 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
예측값: [0. 1. 0.] 결과값: [0. 1. 0.]
정수 인코딩 복원 : [CLS] 삼성증권 새해 첫주 신규고객 4만명 유입 매수 1위는 삼성전자 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PA

정수 인코딩 복원 : [CLS] 日 증시 美 증시 영향으로 상승 닛케이지수 1 78 상승 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
예측값: [0. 0. 1.] 결과값: [0. 0. 1.]
정수 인코딩 복원 : [CLS] 코스닥 시총상위주 출렁 2 넘게 내리며 962선 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PA

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [26]:
print('정수 인코딩 복원 :',tokenizer.decode(test_X[0][6]))

정수 인코딩 복원 : [CLS] 한투증권 테슬라 S P500 무조건 월지급식 ELS 공모 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]


In [48]:
data = '삼성SDI 업황 개선 기대되나 중장기 성장성 약화목표가한국'

In [49]:
target = 1

In [69]:
df = pd.DataFrame([[data, target]], columns=['title','label'])

In [56]:
df1 = pd.DataFrame([data])

In [58]:
df2 = pd.DataFrame([target])

In [62]:
df = pd.concat([df1,df2], axis = 1)
df.columns = ['title','label']

In [70]:
df

,title,label
0,삼성SDI 업황 개선 기대되나 중장기 성장성 약화목표가한국,1


In [64]:
val_X, val_y = convert_examples_to_features(df['title'], df['label'], max_seq_len=128, tokenizer=tokenizer)

  0%|                                                                                            | 0/1 [00:00<?, ?it/s]C:\Users\wjsrn\anaconda3\envs\py39t116\lib\site-packages\transformers\tokenization_utils_base.py:2301: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
100%|████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]


In [65]:
val_y = to_categorical(val_y, 3)

In [66]:
pred = model.predict(val_X)
logits = pred.logits
predictions = np.zeros(logits.shape)
predictions[np.where(logits >= 0.5)] = 1
correct_count = (predictions == val_y).sum().item()/ 3
acc = correct_count / len(test_y) * 100
print(acc)

1/1 [==============================] - 2s 2s/step
0.025


In [67]:
for i in range(len(val_X[0])):
    print('정수 인코딩 복원 :',tokenizer.decode(val_X[0][i]))
    print('예측값: {} 결과값: {}'.format(predictions[i], val_y[i]))

정수 인코딩 복원 : [CLS] 삼성SDI 업황 개선 기대되나 중장기 성장성 약화목표가한국 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
예측값: [0. 1. 0.] 결과값: [0. 1. 0.]
